1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv("Churn_Modelling.csv")
df.drop(columns=["RowNumber", "CustomerId", "Surname"], inplace=True)
df.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
df.shape

(10000, 11)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, :"EstimatedSalary"], df['Exited'], random_state=0)

In [5]:
X_train.shape, X_test.shape

((7500, 10), (2500, 10))

сделаем пайплайн для логистической регрессии, непрервные признаки стандартизируем, дискретные закодируем через OneHot

In [6]:
class CreateDf(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = X.copy()
        return pd.DataFrame(out, columns=self.col_names)

In [7]:
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.get_dummies(X, columns=self.col_names)

In [8]:
standart_scaler_trans = make_column_transformer(
                                                (StandardScaler(), ["CreditScore", "Age", "Balance", 
                                                                    "NumOfProducts", "EstimatedSalary"]),
                                                remainder="passthrough"
                                                )

In [9]:
log_res_trans = make_pipeline(standart_scaler_trans,
                              CreateDf(["CreditScore", "Age", "Balance", "NumOfProducts", "EstimatedSalary",
                                        "Geography", "Gender", "Tenure", "HasCrCard", "IsActiveMember"]))

In [10]:
# посмотрим как отрабатывает трансформер

In [11]:
log_res_trans.fit_transform(X_train).head(3)

,CreditScore,Age,Balance,NumOfProducts,EstimatedSalary,Geography,Gender,Tenure,HasCrCard,IsActiveMember
0,-0.735507,0.0152657,0.67316,2.53503,-1.64081,Germany,Female,5,0,0
1,1.02443,-0.652609,-1.20772,0.804242,-0.0792715,France,Female,5,1,0
2,0.808295,-0.461788,-0.356937,0.804242,-0.99684,Spain,Female,9,1,1


In [12]:
X_train.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
2967,579,Germany,Female,39,5,117833.30,3,0,0,5831.00
700,750,France,Female,32,5,0.00,2,1,0,95611.47
3481,729,Spain,Female,34,9,53299.96,2,1,1,42855.97


In [13]:
#  отрабатывает нормально, фичи не путает, перезапишем его дополнив onehot

In [14]:
log_res_trans = make_pipeline(standart_scaler_trans,
                              CreateDf(["CreditScore", "Age", "Balance", "NumOfProducts", "EstimatedSalary",
                                        "Geography", "Gender", "Tenure", "HasCrCard", "IsActiveMember"]),
                              OHEEncoder(["Geography", "Gender", "Tenure"]))

In [15]:
#  не будет кодировать переменные HasCrCard и IsActiveMember потому что они уже являются бинарными

In [16]:
log_res_trans.fit_transform(X_train).head(3)

,CreditScore,Age,Balance,NumOfProducts,EstimatedSalary,HasCrCard,IsActiveMember,Geography_France,Geography_Germany,Geography_Spain,...,Tenure_1,Tenure_2,Tenure_3,Tenure_4,Tenure_5,Tenure_6,Tenure_7,Tenure_8,Tenure_9,Tenure_10
0,-0.735507,0.0152657,0.67316,2.53503,-1.64081,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1.02443,-0.652609,-1.20772,0.804242,-0.0792715,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0.808295,-0.461788,-0.356937,0.804242,-0.99684,1,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [17]:
#  чтож вроде закодировалось вполне себе нормально, можно попробовать обучить модель логистической регресии и сделать пайп

In [18]:
log_res_solver = LogisticRegression(C=0.6, random_state=42)

In [19]:
final_logres_pipe = make_pipeline(log_res_trans, log_res_solver)

In [20]:
final_logres_pipe.fit(X_train, y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('standardscaler',
                                                                   StandardScaler(),
                                                                   ['CreditScore',
                                                                    'Age',
                                                                    'Balance',
                                                                    'NumOfProducts',
                                                                    'EstimatedSalary'])])),
                                 ('createdf',
                                  CreateDf(col_names=['CreditScore', 'Age',
                                                      'Balance',
                                                      'NumOfProducts

In [21]:
logres_predict = final_logres_pipe.predict_proba(X_test)[:, 1]
logres_predict

array([0.22851385, 0.33462238, 0.15397983, ..., 0.28021655, 0.2496381 ,
       0.12262079])

In [22]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, confusion_matrix

In [23]:
def best_thresold(test, preds):
    precision, recall, thresholds = precision_recall_curve(test, preds)

    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix]))
    return thresholds[ix]

In [24]:
logres_threshold = best_thresold(y_test, logres_predict)

Best Threshold=0.289761, F-Score=0.511, Precision=0.465, Recall=0.568


In [25]:
roc_auc_score(y_test, logres_predict)

0.7721475519997157

не самый лушчший результат, поскольку мы узнаем чуть больше половины реально уходящих клиентов и при этом совершаем еще ошибку второго рода еще большую 1-46,5% = 53,5%. Т.е ошибаемся в том что это за клиент.  Т.е. мы не можем ни нормально определить уходящих, ни обеспечить понимание в разделении уходящих и нет. И вишенкой на торте roc_auc говорящий о том, что модель вполне себе способная. Это связано как раз с дисбалансом классов и дает такой результат, потому что у нас большая оценка по FN получается

Посотрим что будет, при использовании такой модели

3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 

In [26]:
def estimate_economic(test, predict, threshold):
    conf_matrix = confusion_matrix(test, predict>threshold)
    TN = conf_matrix[0, 0]
    FP = conf_matrix[0, 1]
    TP = conf_matrix[1, 1]
    FN = conf_matrix[1, 0]
    total_revenue = TP * 2
    total_expences = (TP + FP) * 1
    net_income = total_revenue - total_expences
    print("Модель зарабатывает {:,}$ при расходах {:,}$. Общая рентабельность {:%}".format(total_revenue, total_expences, 
                                                                                         net_income / total_expences))
    

In [27]:
estimate_economic(y_test, logres_predict, logres_threshold)

Модель зарабатывает 576$ при расходах 621$. Общая рентабельность -7.246377%


Как видно модель хронически убыточная и за такую реализацию оторвут руки))  
Улучшим модель за счет Catboost

In [28]:
from catboost import CatBoostClassifier

In [29]:
cat_solver = CatBoostClassifier(n_estimators=250, depth=7, loss_function="CrossEntropy", 
                                cat_features=["Geography", "Gender", "Tenure", "HasCrCard", "IsActiveMember"],
                                logging_level="Silent")

In [30]:
cat_trans = make_pipeline(standart_scaler_trans,
                          CreateDf(["CreditScore", "Age", "Balance", "NumOfProducts", "EstimatedSalary",
                                    "Geography", "Gender", "Tenure", "HasCrCard", "IsActiveMember"]))

In [31]:
cat_trans.fit_transform(X_train).head(3)

,CreditScore,Age,Balance,NumOfProducts,EstimatedSalary,Geography,Gender,Tenure,HasCrCard,IsActiveMember
0,-0.735507,0.0152657,0.67316,2.53503,-1.64081,Germany,Female,5,0,0
1,1.02443,-0.652609,-1.20772,0.804242,-0.0792715,France,Female,5,1,0
2,0.808295,-0.461788,-0.356937,0.804242,-0.99684,Spain,Female,9,1,1


In [32]:
# вроде трансформер возвращает нормальный датафрейм, едем дальше))

In [33]:
final_cat_pipe = make_pipeline(cat_trans, cat_solver)

In [34]:
final_cat_pipe.fit(X_train, y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('standardscaler',
                                                                   StandardScaler(),
                                                                   ['CreditScore',
                                                                    'Age',
                                                                    'Balance',
                                                                    'NumOfProducts',
                                                                    'EstimatedSalary'])])),
                                 ('createdf',
                                  CreateDf(col_names=['CreditScore', 'Age',
                                                      'Balance',
                                                      'NumOfProducts

In [35]:
cat_predict = final_cat_pipe.predict_proba(X_test)[:, 1]
cat_predict

array([0.29044385, 0.25440594, 0.17354406, ..., 0.19496502, 0.14336466,
       0.06440496])

In [36]:
cat_threshold = best_thresold(y_test, cat_predict)

Best Threshold=0.389082, F-Score=0.660, Precision=0.697, Recall=0.627


In [37]:
roc_auc_score(y_test, cat_predict)

0.8810817638114147

In [38]:
estimate_economic(y_test, cat_predict, cat_threshold)

Модель зарабатывает 636$ при расходах 457$. Общая рентабельность 39.168490%


Ну вот совсем уже другое дело)) чисто на одной модели смогли улучшить результат до нормальной рентабельности. При этом как видим roc_auc поднялся всего лишь на 0.1 где-то. Поэтому в слепую применять такую метрику наврядли нужно при решени таких задач классификации, потому что информативность ее для понимания экономики низкая